In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.neighbors import NearestNeighbors
from scipy import ndarray

In [2]:
filename='preprocessed_majority.csv'
datapd_0=pd.read_csv(filename, index_col=0)

In [3]:
filename='preprocessed_minority.csv'
datapd_1=pd.read_csv(filename, index_col=0 )

In [4]:
print('Majority class dataframe shape:', datapd_0.shape)
print('Minority class dataframe shape:', datapd_1.shape)

Majority class dataframe shape: (8618, 50)
Minority class dataframe shape: (17, 50)


In [5]:
n_feat=datapd_0.shape[1]

In [6]:
print('Imbalance Ratio:', datapd_0.shape[0]/datapd_1.shape[0])

Imbalance Ratio: 506.94117647058823


In [7]:
features_0=np.asarray(datapd_0)
features_1=np.asarray(datapd_1)

In [8]:
s=93
features_1=np.take(features_1,np.random.RandomState(seed=s).permutation(features_1.shape[0]),axis=0,out=features_1)
features_0=np.take(features_0,np.random.RandomState(seed=s).permutation(features_0.shape[0]),axis=0,out=features_0)

In [9]:
a=len(features_1)//3
b=len(features_0)//3

In [10]:
fold_1_min=features_1[0:a]
fold_1_maj=features_0[0:b]
fold_1_tst=np.concatenate((fold_1_min,fold_1_maj))
lab_1_tst=np.concatenate((np.zeros(len(fold_1_min))+1, np.zeros(len(fold_1_maj))))

In [11]:
fold_2_min=features_1[a:2*a]
fold_2_maj=features_0[b:2*b]
fold_2_tst=np.concatenate((fold_2_min,fold_2_maj))
lab_2_tst=np.concatenate((np.zeros(len(fold_1_min))+1, np.zeros(len(fold_1_maj))))

In [12]:
fold_3_min=features_1[2*a:]
fold_3_maj=features_0[2*b:]
fold_3_tst=np.concatenate((fold_3_min,fold_3_maj))
lab_3_tst=np.concatenate((np.zeros(len(fold_3_min))+1, np.zeros(len(fold_3_maj))))

In [13]:
fold_1_trn=np.concatenate((fold_2_min,fold_3_min,fold_2_maj,fold_3_maj))
lab_1_trn=np.concatenate((np.zeros(a+len(fold_3_min))+1,np.zeros(b+len(fold_3_maj))))

In [14]:
fold_2_trn=np.concatenate((fold_1_min,fold_3_min,fold_1_maj,fold_3_maj))
lab_2_trn=np.concatenate((np.zeros(a+len(fold_3_min))+1,np.zeros(b+len(fold_3_maj))))

In [15]:
fold_3_trn=np.concatenate((fold_2_min,fold_1_min,fold_2_maj,fold_1_maj))
lab_3_trn=np.concatenate((np.zeros(2*a)+1,np.zeros(2*b)))

In [16]:
training_folds_feats=[fold_1_trn,fold_2_trn,fold_3_trn]

In [17]:
testing_folds_feats=[fold_1_tst,fold_2_tst,fold_3_tst]

In [18]:
training_folds_labels=[lab_1_trn,lab_2_trn,lab_3_trn]

In [19]:
testing_folds_labels=[lab_1_tst,lab_2_tst,lab_3_tst]

In [20]:
def lr(X_train,y_train,X_test,y_test):
    from sklearn import metrics
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import f1_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import balanced_accuracy_score
    logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial', class_weight={0: 1, 1: 1})
    logreg.fit(X_train, y_train)
    y_pred= logreg.predict(X_test)
    con_mat=confusion_matrix(y_test,y_pred)
    bal_acc=balanced_accuracy_score(y_test,y_pred)
    tn, fp, fn, tp = con_mat.ravel()
    print('tn, fp, fn, tp:', tn, fp, fn, tp)
    f1 = f1_score(y_test, y_pred)
    precision=precision_score(y_test, y_pred)
    recall=recall_score(y_test, y_pred)
    print('balanced accuracy_LR:', bal_acc)
    print('f1 score_LR:', f1)
    print('confusion matrix_LR',con_mat)
    return(f1, bal_acc, precision, recall, con_mat)

In [21]:
def svm(X_train,y_train,X_test,y_test):
    from sklearn import preprocessing
    from sklearn import metrics
    #from sklearn import svm
    from sklearn.svm import LinearSVC
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import f1_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import balanced_accuracy_score
    X_train = preprocessing.scale(X_train)
    X_test = preprocessing.scale(X_test)
    #svm= svm.SVC(kernel='linear', decision_function_shape='ovo', class_weight={0: 1., 1: 1.},probability=True)
    svm= LinearSVC(random_state=0, tol=1e-5)
    svm.fit(X_train, y_train)
    y_pred= svm.predict(X_test)
    con_mat=confusion_matrix(y_test,y_pred)
    bal_acc=balanced_accuracy_score(y_test,y_pred)
    tn, fp, fn, tp = con_mat.ravel()
    print('tn, fp, fn, tp:', tn, fp, fn, tp)
    f1 = f1_score(y_test, y_pred)
    precision=precision_score(y_test, y_pred)
    recall=recall_score(y_test, y_pred)
    print('balanced accuracy_SVM:', bal_acc)
    print('f1 score_SVM:', f1)
    print('confusion matrix_SVM',con_mat)
    return( f1, bal_acc, precision, recall, con_mat)

In [22]:
def knn(X_train,y_train,X_test,y_test):
    from sklearn import metrics
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import f1_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score   
    from sklearn.metrics import balanced_accuracy_score
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(X_train, y_train)
    y_pred= knn.predict(X_test)
    con_mat=confusion_matrix(y_test,y_pred)
    bal_acc=balanced_accuracy_score(y_test,y_pred)
    tn, fp, fn, tp = con_mat.ravel()
    print('tn, fp, fn, tp:', tn, fp, fn, tp)
    print('balanced accuracy_KNN:', bal_acc)
    f1 = f1_score(y_test, y_pred)
    precision=precision_score(y_test, y_pred)
    recall=recall_score(y_test, y_pred)
    print('f1 score_KNN:', f1)
    print('confusion matrix_KNN',con_mat)
    return(f1, bal_acc, precision, recall, con_mat)

In [23]:
def Neb_grps(data,near_neb):
    nbrs = NearestNeighbors(n_neighbors=near_neb, algorithm='ball_tree').fit(data)
    distances, indices = nbrs.kneighbors(data)
    neb_class=[]
    for i in (indices):
        neb_class.append(i)
    return(np.asarray(neb_class)) 

def LoRAS(data,num_samples,shadow,sigma,num_RACOS,num_afcomb):
    np.random.seed(42)
    data_shadow=([])
    for i in range (num_samples):
        c=0
        while c<shadow:
            data_shadow.append(data[i]+np.random.normal(0,sigma))
            c=c+1
    data_shadow==np.asarray(data_shadow)
    data_shadow_lc=([])
    for i in range(num_RACOS):
        idx = np.random.randint(shadow*num_samples, size=num_afcomb)
        w=np.random.randint(100, size=len(idx))
        aff_w=np.asarray(w/sum(w))
        data_tsl=np.array(data_shadow)[idx,:]
        data_tsl_=np.dot(aff_w, data_tsl)
        data_shadow_lc.append(data_tsl_)
    return(np.asarray(data_shadow_lc))   

def LoRAS_gen(num_samples,shadow,sigma,num_RACOS,num_afcomb):
    RACOS_set=[]
    for i in range (len(nb_list)):
        RACOS_i= LoRAS(features_1_trn[nb_list[i]],num_samples,shadow,sigma,num_RACOS,num_afcomb)
        RACOS_set.append(RACOS_i)
    LoRAS_set=np.asarray(RACOS_set)
    LoRAS_1=np.reshape(LoRAS_set,(len(features_1_trn)*num_RACOS,n_feat))
    return(np.concatenate((LoRAS_1,features_1_trn)))

In [24]:
def OVS(training_data,training_labels,neb):
    from imblearn.over_sampling import SMOTE 
    sm = SMOTE(random_state=62, k_neighbors=neb,  kind='regular',ratio=1)
    SMOTE_feat, SMOTE_labels = sm.fit_resample(training_data,training_labels)
    smbl1 = SMOTE(random_state=62, k_neighbors=neb,  kind='borderline1',ratio=1)
    SMOTE_feat_bl1, SMOTE_labels_bl1 = smbl1.fit_resample(training_data,training_labels)
    smbl2 = SMOTE(random_state=62, k_neighbors=neb,  kind='borderline2',ratio=1)
    SMOTE_feat_bl2, SMOTE_labels_bl2 = smbl2.fit_resample(training_data,training_labels)
    smsvm = SMOTE(random_state=62, k_neighbors=neb,  kind='svm',ratio=1)
    SMOTE_feat_svm, SMOTE_labels_svm = smsvm.fit_resample(training_data,training_labels)
    from imblearn.over_sampling import ADASYN
    ad = ADASYN(random_state=62,n_neighbors=neb,  ratio=1)
    ADASYN_feat, ADASYN_labels = ad.fit_resample(training_data,training_labels)
    return(SMOTE_feat, SMOTE_labels,SMOTE_feat_bl1, SMOTE_labels_bl1, SMOTE_feat_bl2, SMOTE_labels_bl2,SMOTE_feat_svm, SMOTE_labels_svm,ADASYN_feat, ADASYN_labels)

In [25]:
LR=[]
SVM=[]
KNN=[] 
LR_SM=[]
SVM_SM=[]
KNN_SM=[]
LR_SMBL1=[]
SVM_SMBL1=[]
KNN_SMBL1=[] 
LR_SMBL2=[]
SVM_SMBL2=[]
KNN_SMBL2=[] 
LR_SMSVM=[]
SVM_SMSVM=[]
KNN_SMSVM=[] 
LR_ADA=[]
SVM_ADA=[]
KNN_ADA=[] 

i=0
while i<3:
    SMOTE_feat, SMOTE_labels,SMOTE_feat_bl1, SMOTE_labels_bl1, SMOTE_feat_bl2, SMOTE_labels_bl2,SMOTE_feat_svm, SMOTE_labels_svm,ADASYN_feat, ADASYN_labels=OVS(training_folds_feats[i],training_folds_labels[i],3)
    
    f1_lr, bal_acc_lr, precision_lr, recall_lr, mat_lr=lr(training_folds_feats[i],training_folds_labels[i],testing_folds_feats[i],testing_folds_labels[i])
    LR.append([f1_lr, bal_acc_lr, precision_lr, recall_lr])
    f1_svm,bal_acc_svm,precision_svm, recall_svm,mat_svm=svm(training_folds_feats[i],training_folds_labels[i],testing_folds_feats[i],testing_folds_labels[i])
    SVM.append([f1_svm,bal_acc_svm,precision_svm, recall_svm])
    f1_knn,bal_acc_knn,precision_knn, recall_knn,mat_knn=knn(training_folds_feats[i],training_folds_labels[i],testing_folds_feats[i],testing_folds_labels[i])
    KNN.append([f1_knn,bal_acc_knn,precision_knn, recall_knn])
    
    f1_lr_SMOTE,bal_acc_lr_SMOTE,precision_lr_SMOTE, recall_lr_SMOTE,mat_lr_SMOTE=lr(SMOTE_feat,SMOTE_labels,testing_folds_feats[i],testing_folds_labels[i])
    LR_SM.append([f1_lr_SMOTE,bal_acc_lr_SMOTE,precision_lr_SMOTE, recall_lr_SMOTE])
    f1_svm_SMOTE,bal_acc_svm_SMOTE,precision_svm_SMOTE, recall_svm_SMOTE,mat_svm_SMOTE=svm(SMOTE_feat,SMOTE_labels,testing_folds_feats[i],testing_folds_labels[i])
    SVM_SM.append([f1_svm_SMOTE,bal_acc_svm_SMOTE,precision_svm_SMOTE, recall_svm_SMOTE])
    f1_knn_SMOTE,bal_acc_knn_SMOTE,precision_knn_SMOTE, recall_knn_SMOTE,mat_knn_SMOTE=knn(SMOTE_feat,SMOTE_labels,testing_folds_feats[i],testing_folds_labels[i])
    KNN_SM.append([f1_knn_SMOTE,bal_acc_knn_SMOTE,precision_knn_SMOTE, recall_knn_SMOTE])
    
    f1_lr_SMOTE_bl1,bal_acc_lr_SMOTE_bl1,precision_lr_SMOTE_bl1, recall_lr_SMOTE_bl1,mat_lr_SMOTE_bl1=lr(SMOTE_feat_bl1,SMOTE_labels_bl1,testing_folds_feats[i],testing_folds_labels[i])
    LR_SMBL1.append([f1_lr_SMOTE_bl1,bal_acc_lr_SMOTE_bl1,precision_lr_SMOTE_bl1, recall_lr_SMOTE_bl1])
    f1_svm_SMOTE_bl1,bal_acc_svm_SMOTE_bl1,precision_svm_SMOTE_bl1, recall_svm_SMOTE_bl1,mat_svm_SMOTE_bl1=svm(SMOTE_feat_bl1,SMOTE_labels_bl1,testing_folds_feats[i],testing_folds_labels[i])
    SVM_SMBL1.append([f1_svm_SMOTE_bl1,bal_acc_svm_SMOTE_bl1,precision_svm_SMOTE_bl1, recall_svm_SMOTE_bl1])
    f1_knn_SMOTE_bl1,bal_acc_knn_SMOTE_bl1,precision_knn_SMOTE_bl1, recall_knn_SMOTE_bl1,mat_knn_SMOTE_bl1=knn(SMOTE_feat_bl1,SMOTE_labels_bl1,testing_folds_feats[i],testing_folds_labels[i])
    KNN_SMBL1.append([f1_knn_SMOTE_bl1,bal_acc_knn_SMOTE_bl1,precision_knn_SMOTE_bl1, recall_knn_SMOTE_bl1])
    
    
    f1_lr_SMOTE_bl2,bal_acc_lr_SMOTE_bl2,precision_lr_SMOTE_bl2, recall_lr_SMOTE_bl2,mat_lr_SMOTE_bl2=lr(SMOTE_feat_bl2,SMOTE_labels_bl2,testing_folds_feats[i],testing_folds_labels[i])
    LR_SMBL2.append([f1_lr_SMOTE_bl2,bal_acc_lr_SMOTE_bl2,precision_lr_SMOTE_bl2, recall_lr_SMOTE_bl2])
    f1_svm_SMOTE_bl2,bal_acc_svm_SMOTE_bl2,precision_svm_SMOTE_bl2, recall_svm_SMOTE_bl2,mat_svm_SMOTE_bl2=svm(SMOTE_feat_bl1,SMOTE_labels_bl1,testing_folds_feats[i],testing_folds_labels[i])
    SVM_SMBL2.append([f1_svm_SMOTE_bl2,bal_acc_svm_SMOTE_bl2,precision_svm_SMOTE_bl2, recall_svm_SMOTE_bl2])
    f1_knn_SMOTE_bl2,bal_acc_knn_SMOTE_bl2,precision_knn_SMOTE_bl2, recall_knn_SMOTE_bl2,mat_knn_SMOTE_bl2=knn(SMOTE_feat_bl2,SMOTE_labels_bl2,testing_folds_feats[i],testing_folds_labels[i])
    KNN_SMBL2.append([f1_knn_SMOTE_bl2,bal_acc_knn_SMOTE_bl2,precision_knn_SMOTE_bl2, recall_knn_SMOTE_bl2])
    
    f1_lr_SMOTE_svm,bal_acc_lr_SMOTE_svm,precision_lr_SMOTE_svm, recall_lr_SMOTE_svm,mat_lr_SMOTE_svm=lr(SMOTE_feat_svm,SMOTE_labels_svm,testing_folds_feats[i],testing_folds_labels[i])
    LR_SMSVM.append([f1_lr_SMOTE_svm,bal_acc_lr_SMOTE_svm,precision_lr_SMOTE_svm, recall_lr_SMOTE_svm])
    f1_svm_SMOTE_svm,bal_acc_svm_SMOTE_svm,precision_svm_SMOTE_svm, recall_svm_SMOTE_svm,mat_svm_SMOTE_svm=svm(SMOTE_feat_svm,SMOTE_labels_svm,testing_folds_feats[i],testing_folds_labels[i])
    SVM_SMSVM.append([f1_svm_SMOTE_svm,bal_acc_svm_SMOTE_svm,precision_svm_SMOTE_svm, recall_svm_SMOTE_svm])
    f1_knn_SMOTE_svm,bal_acc_knn_SMOTE_svm,precision_knn_SMOTE_svm, recall_knn_SMOTE_svm,mat_knn_SMOTE_svm=knn(SMOTE_feat_svm,SMOTE_labels_svm,testing_folds_feats[i],testing_folds_labels[i])
    KNN_SMSVM.append([f1_knn_SMOTE_svm,bal_acc_knn_SMOTE_svm,precision_knn_SMOTE_svm, recall_knn_SMOTE_svm])
    
    f1_lr_ADASYN,bal_acc_lr_ADASYN,precision_lr_ADASYN, recall_lr_ADASYN,mat_lr_ADASYN=lr(ADASYN_feat,ADASYN_labels,testing_folds_feats[i],testing_folds_labels[i])
    LR_ADA.append([f1_lr_ADASYN,bal_acc_lr_ADASYN,precision_lr_ADASYN, recall_lr_ADASYN])
    f1_svm_ADASYN,bal_acc_svm_ADASYN,precision_svm_ADASYN, recall_svm_ADASYN,mat_svm_ADASYN=svm(ADASYN_feat,ADASYN_labels,testing_folds_feats[i],testing_folds_labels[i])
    SVM_ADA.append([f1_svm_ADASYN,bal_acc_svm_ADASYN,precision_svm_ADASYN, recall_svm_ADASYN])
    f1_knn_ADASYN,bal_acc_knn_ADASYN,precision_knn_ADASYN, recall_knn_ADASYN,mat_knn_ADASYN=knn(ADASYN_feat,ADASYN_labels,testing_folds_feats[i],testing_folds_labels[i])
    KNN_ADA.append([f1_knn_ADASYN,bal_acc_knn_ADASYN,precision_knn_ADASYN, recall_knn_ADASYN])
    
    i=i+1

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in

tn, fp, fn, tp: 2871 1 0 5
balanced accuracy_LR: 0.9998259052924792
f1 score_LR: 0.9090909090909091
confusion matrix_LR [[2871    1]
 [   0    5]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


tn, fp, fn, tp: 2871 1 1 4
balanced accuracy_SVM: 0.8998259052924791
f1 score_SVM: 0.8000000000000002
confusion matrix_SVM [[2871    1]
 [   1    4]]
tn, fp, fn, tp: 2872 0 2 3
balanced accuracy_KNN: 0.8
f1 score_KNN: 0.7499999999999999
confusion matrix_KNN [[2872    0]
 [   2    3]]
tn, fp, fn, tp: 2871 1 0 5
balanced accuracy_LR: 0.9998259052924792
f1 score_LR: 0.9090909090909091
confusion matrix_LR [[2871    1]
 [   0    5]]
tn, fp, fn, tp: 1828 1044 0 5
balanced accuracy_SVM: 0.8182451253481895
f1 score_SVM: 0.009487666034155597
confusion matrix_SVM [[1828 1044]
 [   0    5]]
tn, fp, fn, tp: 2871 1 0 5
balanced accuracy_KNN: 0.9998259052924792
f1 score_KNN: 0.9090909090909091
confusion matrix_KNN [[2871    1]
 [   0    5]]
tn, fp, fn, tp: 2871 1 0 5
balanced accuracy_LR: 0.9998259052924792
f1 score_LR: 0.9090909090909091
confusion matrix_LR [[2871    1]
 [   0    5]]
tn, fp, fn, tp: 1939 933 0 5
balanced accuracy_SVM: 0.8375696378830084
f1 score_SVM: 0.010604453870625663
confusion 

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


...

tn, fp, fn, tp: 2874 0 3 4
balanced accuracy_SVM: 0.7857142857142857
f1 score_SVM: 0.7272727272727273
confusion matrix_SVM [[2874    0]
 [   3    4]]
tn, fp, fn, tp: 2874 0 1 6
balanced accuracy_KNN: 0.9285714285714286
f1 score_KNN: 0.923076923076923
confusion matrix_KNN [[2874    0]
 [   1    6]]
tn, fp, fn, tp: 2874 0 1 6
balanced accuracy_LR: 0.9285714285714286
f1 score_LR: 0.923076923076923
confusion matrix_LR [[2874    0]
 [   1    6]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


tn, fp, fn, tp: 1946 928 0 7
balanced accuracy_SVM: 0.8385525400139179
f1 score_SVM: 0.014861995753715497
confusion matrix_SVM [[1946  928]
 [   0    7]]
tn, fp, fn, tp: 2873 1 0 7
balanced accuracy_KNN: 0.9998260264439804
f1 score_KNN: 0.9333333333333333
confusion matrix_KNN [[2873    1]
 [   0    7]]
tn, fp, fn, tp: 2874 0 0 7
balanced accuracy_LR: 1.0
f1 score_LR: 1.0
confusion matrix_LR [[2874    0]
 [   0    7]]
tn, fp, fn, tp: 2105 769 0 7
balanced accuracy_SVM: 0.866214335421016
f1 score_SVM: 0.017879948914431672
confusion matrix_SVM [[2105  769]
 [   0    7]]
tn, fp, fn, tp: 2873 1 0 7
balanced accuracy_KNN: 0.9998260264439804
f1 score_KNN: 0.9333333333333333
confusion matrix_KNN [[2873    1]
 [   0    7]]
tn, fp, fn, tp: 2874 0 0 7
balanced accuracy_LR: 1.0
f1 score_LR: 1.0
confusion matrix_LR [[2874    0]
 [   0    7]]
tn, fp, fn, tp: 2105 769 0 7
balanced accuracy_SVM: 0.866214335421016
f1 score_SVM: 0.017879948914431672
confusion matrix_SVM [[2105  769]
 [   0    7]]
tn, fp

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)



f1 score_SVM: 0.018970189701897018
confusion matrix_SVM [[2150  724]
 [   0    7]]
tn, fp, fn, tp: 2873 1 0 7
balanced accuracy_KNN: 0.9998260264439804
f1 score_KNN: 0.9333333333333333
confusion matrix_KNN [[2873    1]
 [   0    7]]
tn, fp, fn, tp: 2874 0 0 7
balanced accuracy_LR: 1.0
f1 score_LR: 1.0
confusion matrix_LR [[2874    0]
 [   0    7]]
tn, fp, fn, tp: 2043 831 0 7
balanced accuracy_SVM: 0.855427974947808
f1 score_SVM: 0.016568047337278107
confusion matrix_SVM [[2043  831]
 [   0    7]]
tn, fp, fn, tp: 2873 1 0 7
balanced accuracy_KNN: 0.9998260264439804
f1 score_KNN: 0.9333333333333333
confusion matrix_KNN [[2873    1]
 [   0    7]]


In [26]:
LR_LoRAS=[]
SVM_LoRAS=[]
KNN_LoRAS=[]
for i in range(3):
    
    features = training_folds_feats[i]
    labels= training_folds_labels[i]
    label_1=np.where(labels == 1)[0]
    label_1=list(label_1)
    features_1_trn=features[label_1]
    
    label_0=np.where(labels == 0)[0]
    label_0=list(label_0)
    features_0_trn=features[label_0]
    
    num_samples=3 
    shadow=100
    sigma=.005
    num_RACOS=(len(features_0_trn)-len(features_1_trn))//len(features_1_trn)
    num_afcomb=50
    nb_list=Neb_grps(features_1_trn, num_samples)
    
    LoRAS_1=LoRAS_gen(num_samples,shadow,sigma,num_RACOS,num_afcomb)
    LoRAS_train=np.concatenate((LoRAS_1,features_0_trn))
    LoRAS_labels=np.concatenate((np.zeros(len(LoRAS_1))+1, np.zeros(len(features_0_trn))))
    
    f1_lr_LoRAS,bal_acc_lr_LoRAS,precision_lr_LoRAS, recall_lr_LoRAS,mat_lr_LoRAS=lr(LoRAS_train,LoRAS_labels,testing_folds_feats[i],testing_folds_labels[i])
    LR_LoRAS.append([f1_lr_LoRAS,bal_acc_lr_LoRAS,precision_lr_LoRAS, recall_lr_LoRAS])
    f1_svm_LoRAS,bal_acc_svm_LoRAS,precision_svm_LoRAS, recall_svm_LoRAS,mat_svm_LoRAS=svm(LoRAS_train,LoRAS_labels,testing_folds_feats[i],testing_folds_labels[i])
    SVM_LoRAS.append([f1_svm_LoRAS,bal_acc_svm_LoRAS,precision_svm_LoRAS, recall_svm_LoRAS])
    f1_knn_LoRAS,bal_acc_knn_LoRAS,precision_knn_LoRAS, recall_knn_LoRAS,mat_knn_LoRAS=knn(LoRAS_train,LoRAS_labels,testing_folds_feats[i],testing_folds_labels[i])
    KNN_LoRAS.append([f1_knn_LoRAS,bal_acc_knn_LoRAS,precision_knn_LoRAS, recall_knn_LoRAS])    

tn, fp, fn, tp: 2871 1 0 5
balanced accuracy_LR: 0.9998259052924792
f1 score_LR: 0.9090909090909091
confusion matrix_LR [[2871    1]
 [   0    5]]
tn, fp, fn, tp: 1806 1066 0 5
balanced accuracy_SVM: 0.8144150417827298
f1 score_SVM: 0.00929368029739777
confusion matrix_SVM [[1806 1066]
 [   0    5]]
tn, fp, fn, tp: 2871 1 0 5
balanced accuracy_KNN: 0.9998259052924792
f1 score_KNN: 0.9090909090909091
confusion matrix_KNN [[2871    1]
 [   0    5]]
tn, fp, fn, tp: 2872 0 0 5
balanced accuracy_LR: 1.0
f1 score_LR: 1.0
confusion matrix_LR [[2872    0]
 [   0    5]]
tn, fp, fn, tp: 1925 947 0 5
balanced accuracy_SVM: 0.8351323119777159
f1 score_SVM: 0.01044932079414838
confusion matrix_SVM [[1925  947]
 [   0    5]]
tn, fp, fn, tp: 2872 0 0 5
balanced accuracy_KNN: 1.0
f1 score_KNN: 1.0
confusion matrix_KNN [[2872    0]
 [   0    5]]
tn, fp, fn, tp: 2874 0 1 6
balanced accuracy_LR: 0.9285714285714286
f1 score_LR: 0.923076923076923
confusion matrix_LR [[2874    0]
 [   1    6]]
tn, fp, fn, t

In [27]:
LR_tLoRAS=[]
SVM_tLoRAS=[]
KNN_tLoRAS=[]
from sklearn.manifold import TSNE
for i in range(3):
    
    features = training_folds_feats[i]
    labels= training_folds_labels[i]
    label_1=np.where(labels == 1)[0]
    label_1=list(label_1)
    features_1_trn=features[label_1]
    
    label_0=np.where(labels == 0)[0]
    label_0=list(label_0)
    features_0_trn=features[label_0]
    
    data_embedded_min = TSNE().fit_transform(features_1_trn)
    result_min= pd.DataFrame(data = data_embedded_min, columns = ['t-SNE0', 't-SNE1'])
    min_t=np.asmatrix(result_min)
    min_t=min_t[0:len(features_1_trn)]
    min_t=min_t[:, [0,1]]
    
    num_samples=3 
    shadow=100
    sigma=.005
    num_RACOS=(len(features_0_trn)-len(features_1_trn))//len(features_1_trn)
    num_afcomb=50
    nb_list=Neb_grps(min_t, num_samples)
    
    LoRAS_1=LoRAS_gen(num_samples,shadow,sigma,num_RACOS,num_afcomb)
    LoRAS_train=np.concatenate((LoRAS_1,features_0_trn))
    LoRAS_labels=np.concatenate((np.zeros(len(LoRAS_1))+1, np.zeros(len(features_0_trn))))
    
    f1_lr_LoRAS,bal_acc_lr_LoRAS,precision_lr_LoRAS, recall_lr_LoRAS,mat_lr_LoRAS=lr(LoRAS_train,LoRAS_labels,testing_folds_feats[i],testing_folds_labels[i])
    LR_tLoRAS.append([f1_lr_LoRAS,bal_acc_lr_LoRAS,precision_lr_LoRAS, recall_lr_LoRAS])
    f1_svm_LoRAS,bal_acc_svm_LoRAS,precision_svm_LoRAS, recall_svm_LoRAS,mat_svm_LoRAS=svm(LoRAS_train,LoRAS_labels,testing_folds_feats[i],testing_folds_labels[i])
    SVM_tLoRAS.append([f1_svm_LoRAS,bal_acc_svm_LoRAS,precision_svm_LoRAS, recall_svm_LoRAS])
    f1_knn_LoRAS,bal_acc_knn_LoRAS,precision_knn_LoRAS, recall_knn_LoRAS,mat_knn_LoRAS=knn(LoRAS_train,LoRAS_labels,testing_folds_feats[i],testing_folds_labels[i])
    KNN_tLoRAS.append([f1_knn_LoRAS,bal_acc_knn_LoRAS,precision_knn_LoRAS, recall_knn_LoRAS])    

tn, fp, fn, tp: 2871 1 0 5
balanced accuracy_LR: 0.9998259052924792
f1 score_LR: 0.9090909090909091
confusion matrix_LR [[2871    1]
 [   0    5]]
tn, fp, fn, tp: 1915 957 0 5
balanced accuracy_SVM: 0.833391364902507
f1 score_SVM: 0.010341261633919338
confusion matrix_SVM [[1915  957]
 [   0    5]]
tn, fp, fn, tp: 2871 1 0 5
balanced accuracy_KNN: 0.9998259052924792
f1 score_KNN: 0.9090909090909091
confusion matrix_KNN [[2871    1]
 [   0    5]]
tn, fp, fn, tp: 2872 0 0 5
balanced accuracy_LR: 1.0
f1 score_LR: 1.0
confusion matrix_LR [[2872    0]
 [   0    5]]
tn, fp, fn, tp: 1876 996 0 5
balanced accuracy_SVM: 0.8266016713091922
f1 score_SVM: 0.009940357852882704
confusion matrix_SVM [[1876  996]
 [   0    5]]
tn, fp, fn, tp: 2872 0 0 5
balanced accuracy_KNN: 1.0
f1 score_KNN: 1.0
confusion matrix_KNN [[2872    0]
 [   0    5]]
tn, fp, fn, tp: 2874 0 1 6
balanced accuracy_LR: 0.9285714285714286
f1 score_LR: 0.923076923076923
confusion matrix_LR [[2874    0]
 [   1    6]]
tn, fp, fn, t

In [28]:
def stats(arr):
    x=np.mean(np.asarray(arr), axis = 0)
    y=np.std(np.asarray(arr), axis = 0)
    return(x,y)

In [29]:

print('F1|Balanced Accuracy|precision|recall :: mean|sd')
print('Without Oversampling')

LR_m, LR_sd=stats(LR)
print('lr:',LR_m, LR_sd)
SVM_m, SVM_sd=stats(SVM)
print('svm:',SVM_m, SVM_sd)
KNN_m, KNN_sd= stats(KNN)
print('knn:',KNN_m, KNN_sd)

print('SMOTE Oversampling')
LR_SM_m, LR_SM_sd=stats(LR_SM)
print('lr:',LR_SM_m, LR_SM_sd)
SVM_SM_m, SVM_SM_sd=stats(SVM_SM)
print('svm:',SVM_SM_m, SVM_SM_sd)
KNN_SM_m, KNN_SM_sd=stats(KNN_SM)
print('knn:',KNN_SM_m, KNN_SM_sd)

print('SMOTE-Bl1 Oversampling')
LR_SMBL1_m, LR_SMBL1_sd=stats(LR_SMBL1)
print('lr:',LR_SMBL1_m, LR_SMBL1_sd)
SVM_SMBL1_m,SVM_SMBL1_sd=stats(SVM_SMBL1)
print('svm:',SVM_SMBL1_m,SVM_SMBL1_sd)
KNN_SMBL1_m,  KNN_SMBL1_sd= stats(KNN_SMBL1)
print('knn:',KNN_SMBL1_m,  KNN_SMBL1_sd)

print('SMOTE-Bl2 Oversampling')
LR_SMBL2_m, LR_SMBL2_sd=stats(LR_SMBL2)
print('lr:',LR_SMBL2_m, LR_SMBL2_sd)
SVM_SMBL2_m, SVM_SMBL2_sd=stats(SVM_SMBL2)
print('svm:',SVM_SMBL2_m, SVM_SMBL2_sd)
KNN_SMBL2_m, KNN_SMBL2_sd= stats(KNN_SMBL2)
print('knn:',KNN_SMBL2_m, KNN_SMBL2_sd)

print('SMOTE-SVM Oversampling')
LR_SMSVM_m, LR_SMSVM_sd=stats(LR_SMSVM)
print('lr:',LR_SMSVM_m, LR_SMSVM_sd)
SVM_SMSVM_m, SVM_SMSVM_sd=stats(SVM_SMSVM)
print('svm:',SVM_SMSVM_m, SVM_SMSVM_sd)
KNN_SMSVM_m, KNN_SMSVM_sd= stats(KNN_SMSVM)
print('knn:',KNN_SMSVM_m, KNN_SMSVM_sd)

print('ADASYN Oversampling')
LR_ADA_m, LR_ADA_sd=stats(LR_ADA)
print('lr:',LR_ADA_m, LR_ADA_sd)
SVM_ADA_m, SVM_ADA_sd=stats(SVM_ADA)
print('svm:',SVM_ADA_m, SVM_ADA_sd)
KNN_ADA_m, KNN_ADA_sd=stats(KNN_ADA)
print('knn:',KNN_ADA_m, KNN_ADA_sd)

print('LoRAS Oversampling')
LR_LoRAS_m, LR_LoRAS_sd=stats(LR_LoRAS)
print('lr:',LR_LoRAS_m, LR_LoRAS_sd)
SVM_LoRAS_m, SVM_LoRAS_sd=stats(SVM_LoRAS)
print('svm:',SVM_LoRAS_m, SVM_LoRAS_sd)
KNN_LoRAS_m, KNN_LoRAS_sd=stats(KNN_LoRAS)
print('knn:',KNN_LoRAS_m, KNN_LoRAS_sd)

print('tLoRAS Oversampling')
LR_tLoRAS_m, LR_tLoRAS_sd=stats(LR_tLoRAS)
print('lr:',LR_tLoRAS_m, LR_tLoRAS_sd)
SVM_tLoRAS_m, SVM_tLoRAS_sd=stats(SVM_tLoRAS)
print('svm:',SVM_tLoRAS_m, SVM_tLoRAS_sd)
KNN_tLoRAS_m, KNN_tLoRAS_sd=stats(KNN_tLoRAS)
print('knn:',KNN_tLoRAS_m, KNN_tLoRAS_sd)

F1|Balanced Accuracy|precision|recall :: mean|sd
Without Oversampling
lr: [0.90701891 0.94279911 0.94444444 0.88571429] [0.0140339  0.04197716 0.07856742 0.08411201]
svm: [0.80538721 0.86184673 0.93333333 0.72380952] [0.0660894  0.05383381 0.0942809  0.1077496 ]
knn: [0.89102564 0.90952381 1.         0.81904762] [0.10454812 0.08275308 0.         0.16550616]
SMOTE Oversampling
lr: [0.94405594 0.97613244 0.94444444 0.95238095] [0.03996836 0.03363079 0.07856742 0.0673435 ]
svm: [0.01172016 0.83250034 0.00589595 1.        ] [0.00228634 0.01011813 0.0011574  0.        ]
knn: [0.94747475 0.99988398 0.90277778 1.        ] [3.84369648e-02 8.20404914e-05 7.08197155e-02 0.00000000e+00]
SMOTE-Bl1 Oversampling
lr: [0.96969697 0.99994197 0.94444444 1.        ] [4.28549564e-02 8.20690322e-05 7.85674201e-02 0.00000000e+00]
svm: [0.01297064 0.84618937 0.00653073 1.        ] [0.00347215 0.01420538 0.00176099 0.        ]
knn: [0.94747475 0.99988398 0.90277778 1.        ] [3.84369648e-02 8.20404914e-05 7

In [30]:
!jupyter nbconvert --to html CV.ipynb

[NbConvertApp] Converting notebook CV.ipynb to html
[NbConvertApp] Writing 399535 bytes to CV.html
